In [1]:
import os 
from pathlib import Path
from typing import Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
import torchaudio

import pandas as pd
import numpy as np

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
class SarcasmDataset(Dataset):
    
    def __init__(self, file_path: str):
        self.file_path=file_path 
        
        walker = sorted(str(p) for p in Path(self.file_path).glob(f'*/*'))
        self.walker = list(walker)
        
        
    def __getitem__(self, n: int) -> Tuple[torch.Tensor, str]:
        fileid = self.walker[n]
        label= self.walker[n].split('/')[-2]
        
        relpath = os.path.relpath(fileid, self.file_path)
        label, filename = os.path.split(relpath)
        waveform, sample_rate = torchaudio.load(fileid)
        
        return waveform, label
    
    def __len__(self) -> int:
        return len(self.walker)

In [4]:
data=SarcasmDataset('../Audios')

In [5]:
train_len=int(len(data)*0.8)
val_len=int(len(data)*0.2)

train_set, val_set=torch.utils.data.random_split(data, [train_len, val_len+1])

In [6]:
kwargs = {'num_workers': 1, 'pin_memory': True} if device == 'cuda' else {}

train_loader = torch.utils.data.DataLoader(train_set,shuffle = True, **kwargs)
val_loader = torch.utils.data.DataLoader(val_set,shuffle = True, **kwargs)